In [1]:
import time

import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import pandas as pd
from tqdm import tqdm

d:\GitHub\research\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# https://aistudio.google.com/app/apikey
api_key = '<YOUR_API_KEY>'
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-1.5-flash')

df = pd.read_csv('LK_modified.xlsx - Вопрос ответ.csv')
content_counts = df['content'].value_counts()
df = df[df['content'].isin(content_counts[content_counts <= 5].index)]

In [3]:
for id, question, content, category in tqdm(
    df[['id', 'question', 'content', 'category']].values
):
    synthetic_data = []

    for i in range(5):
        request = f"""
            Вот несколько примеров вопросов, которые обычно задают во внутреннюю текстовую поддержку, и заранее подготовленных типовых ответов на них:

            1. Вопрос: больничный по уходу за родственником
            Ответ (фиксированный): Больничный с кодом 09 (уход за больным членом семьи) оплачивается полностью за счет средств СФР, работодатель в данном случае выплату не производит.

            2. Вопрос: эцп не активка
            Ответ (фиксированный): Выпуск УНЭП ЮЛ осуществляется в Личный кабинете - Блок «Электронные подписи». Данный блок можно найти в правом верхнем углу, в выпадающем списке под своими ФИО. На странице Вы найдете соответствующую плашку. На данной плашке необходимо нажать «Получить электронную подпись». Подробная инструкция доступна по ссылке

            3. Вопрос: мне нужно изменить % занятости
            Ответ (фиксированный): Для изменения процента занятости сотруднику создайте, пожалуйста, заявку на сотрудника по теме 'Изменение режима, характера работы'


            Далее тебе нужно сгененировать 3 вопроса, релевантных приведенному ниже ответу. Обрати внимание, что сам ответ генерировать тебе не нужно, а вопросы могут иметь орфографические и пунктуационные ошибки, начинаться с маленькой буквы или иметь другие огрехи текстового ввода.

            Вопрос: {question}
            Ответ (фиксированный, его не нужно генерировать): {content}

            Приведи в пример еще 3 вопроса, на которые был бы ответ выше, записав их в отдельные строки через символ \n без дополнительной информации, перечисления цифрами или использования черточек:
        """
        response = model.generate_content(
            request,
            safety_settings={
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
            },
        ).text

        for example in response.split('\n'):
            if example:
                synthetic_data.append((example, content, category))

        time.sleep(6)

    synthetic_df = pd.DataFrame(
        synthetic_data, columns=['question', 'content', 'category']
    )
    synthetic_df.to_csv(f'./synthetic_data/synthetic_{id}.csv', index=False)
    synthetic_df.tail()

100%|██████████| 329/329 [20:31<00:00,  3.74s/it]


In [4]:
import os


directory_path = './synthetic_data/'
synthetic_df = pd.DataFrame()

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)

        synthetic_df = pd.concat(
            [
                synthetic_df,
                pd.read_csv(file_path),
            ],
            ignore_index=True,
        )

print(synthetic_df.shape)
synthetic_df['question'] = synthetic_df['question'].str.strip()
synthetic_df.drop_duplicates(subset='question', inplace=True)
synthetic_df = synthetic_df.reset_index(drop=True)
synthetic_df.to_csv('synthetic_df.csv', index=False)
synthetic_df.tail()

(4937, 3)


,question,content,category
4097,как сделать продление срочного трудового договора,Для продления срочного трудового договора созд...,график работы
4098,как продлить договор,Для продления срочного трудового договора созд...,график работы
4099,как перевести сотрудника на бессрочный ТД,Для продления срочного трудового договора созд...,график работы
4100,как продлить трудовой договор?,Для продления срочного трудового договора созд...,график работы
4101,можно ли продлить срочный трудовой договор?,Для продления срочного трудового договора созд...,график работы
